In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
gc.collect()

官方指認欄位 = [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]
官方指認欄位 = [str(i) for i in 官方指認欄位]

df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
test_data = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
test_data.head()

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


# 根據apriori關聯分析演算法建立對應表,shop_tag映射到其他shop_tag的映射

In [7]:
from apyori import apriori
n = 300
data = []
for chid in tqdm(df.sample(n)['chid'].values):
    data.append(df.loc[df.chid==chid,'shop_tag'].value_counts().index.tolist())
association_rules = apriori(data, min_support=0.10, min_confidence=0.2, min_lift=1.2,max_length=2) 
association_results = list(association_rules)
for item in association_results[:5]:
   pair = item[0] 
   items = [x for x in pair]
   print("Rule: " + items[0] + " -> " + items[1])
   print("Support: " + str(item[1]))
   print("Confidence: " + str(item[2][0][2]))
   print("Lift: " + str(item[2][0][3]))
   print("=====================================")

100%|██████████| 300/300 [00:07<00:00, 41.56it/s]


Rule: 10 -> 13
Support: 0.30666666666666664
Confidence: 0.4070796460176991
Lift: 1.2991903596309544
Rule: 10 -> 3
Support: 0.2
Confidence: 0.26548672566371684
Lift: 1.2067578439259856
Rule: 10 -> 31
Support: 0.43
Confidence: 0.5707964601769911
Lift: 1.205908014458432
Rule: 10 -> 38
Support: 0.17666666666666667
Confidence: 0.2345132743362832
Lift: 1.3028515240904621
Rule: 10 -> 44
Support: 0.13666666666666666
Confidence: 0.976190476190476
Lift: 1.29582806573957


In [8]:
mapping = {}
for key in 官方指認欄位:
   mapping[key] = []
for item in association_results:
   pair = item[0] 
   items = [x for x in pair]
   source = items[0]
   target = items[1]
   if (source in 官方指認欄位) and (target in 官方指認欄位):
       mapping[source].append(target)
mapping

{'2': [],
 '6': ['13'],
 '10': ['13'],
 '12': ['13', '26'],
 '13': [],
 '15': [],
 '18': ['13', '21', '25', '39'],
 '19': [],
 '21': ['13', '39', '6'],
 '22': ['13', '18', '25'],
 '25': ['13'],
 '26': ['25', '6'],
 '36': ['13'],
 '37': [],
 '39': ['6'],
 '48': ['13']}

In [4]:
import joblib
joblib.dump(mapping,'../data/shop_tag_mapping.pkl')

['../data/shop_tag_mapping.pkl']

In [5]:
mapping = joblib.load('../data/shop_tag_mapping.pkl')
mapping

{'2': [],
 '6': [],
 '10': ['13'],
 '12': ['13', '39'],
 '13': [],
 '15': [],
 '18': ['13', '39'],
 '19': [],
 '21': ['13', '6'],
 '22': ['13', '18', '21', '25'],
 '25': ['13'],
 '26': [],
 '36': ['13'],
 '37': [],
 '39': ['13', '6'],
 '48': []}

In [6]:
n = 420
predicts = []
for chid in tqdm(test_data['chid'].values[:n]):
    a = df.loc[df['chid']==chid,'shop_tag'].value_counts().to_frame() # 根據chid對shop_tag做value_counts()
    a['在指認欄位'] = False
    if len(list(set(a.index)&set(官方指認欄位))) != 0: # 判斷這個chid出現過的shop_tag跟官方指認欄位是否有交集
        a.loc[list(set(a.index)&set(官方指認欄位)),'在指認欄位'] = True
    answer = a[a['在指認欄位']==True].index.tolist()[:3] # 取value_counts()前三大的當作答案
    if len(answer) == 3:# 如果成功取出前三大的predicts直接指派answer
        predicts.append(answer)
    else: # 如果沒有找到三個的話
        for _ in range(3-len(answer)):
            if len(answer) == 0: # 如果找到0個 則隨機指派
                answer.append(np.random.choice(官方指認欄位))
            else: # 否則建立候選清單
                tag_list = mapping[np.random.choice(answer)]
                if len(tag_list) > 0: # 如果候選清單大於0
                    answer.append(np.random.choice(tag_list)) # 從候選清單挑
                else: # 否則從官方指認欄位挑
                    answer.append(np.random.choice(官方指認欄位))
        predicts.append(answer)
submission = test_data.copy()
submission.iloc[:n,-3:] = np.array(predicts)
submission.head()

100%|██████████| 420/420 [00:12<00:00, 34.49it/s]


,chid,top1,top2,top3
0,10128239,37,25,22
1,10077943,36,13,15
2,10277876,15,36,10
3,10364842,37,36,10
4,10392717,48,15,12
